# 0) Importing the right tools and data for prediction <font color='blue'> (10 min) </font>

### <font color='red'>0.1) Import the necessary packages with their usual aliases: </font>

- pandas as pd
- numpy as np
- seaborn as sns
- matplotlib.pyplot as plt

In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### IMPORT THE USUAL PACKAGES WITH THEIR ALIASES ####

import warnings
warnings.filterwarnings('ignore')

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/My Drive/dataset_gclass
!ls

Mounted at /content/drive
/content/drive/My Drive/dataset_gclass
 201507-citibike-tripdata.csv	     'input_data_for_modeling_purpose (1).csv'
 201508-citibike-tripdata.csv	      logisticregression.pkl
 data_after_feature_engineering.csv   randomforest.pkl


### <font color='red'>0.1) Download July 2015 NYC Data </font>

You need to download data (July 2015) from https://www.citibikenyc.com/system-data and has been open-sourced by *City Bike*.

The dataset consists in all bike trips in NYC that happened in July 2015, with the following information:
- Trip Duration (seconds)
- Start Time and Date
- Stop Time and Date
- Start Station Name
- End Station Name
- Station ID 
- Station Lat/Long
- Bike ID
- User Type (Customer = 24-hour pass or 7-day pass user; Subscriber = Annual Member)
- Gender (Zero=unknown; 1=male; 2=female)
- Year of Birth

### <font color='red'>0.2) Import the dataset from <i>'../data/201507-citibike-tripdata.csv'</i></font>

In [3]:
import os
os.getcwd()

'/content/drive/My Drive/dataset_gclass'

In [4]:
raw_data = pd.read_csv('201508-citibike-tripdata.csv')
data = raw_data.copy()

In [45]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179044 entries, 0 to 1179043
Data columns (total 15 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   tripduration             1179044 non-null  int64  
 1   starttime                1179044 non-null  object 
 2   stoptime                 1179044 non-null  object 
 3   start station id         1179044 non-null  int64  
 4   start station name       1179044 non-null  object 
 5   start station latitude   1179044 non-null  float64
 6   start station longitude  1179044 non-null  float64
 7   end station id           1179044 non-null  int64  
 8   end station name         1179044 non-null  object 
 9   end station latitude     1179044 non-null  float64
 10  end station longitude    1179044 non-null  float64
 11  bikeid                   1179044 non-null  int64  
 12  usertype                 1179044 non-null  object 
 13  birth year               958043 non-null  

In [5]:
raw_data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1202,8/1/2015 00:00:04,8/1/2015 00:20:07,168,W 18 St & 6 Ave,40.739713,-73.994564,385,E 55 St & 2 Ave,40.757973,-73.966033,23253,Subscriber,1987.0,1
1,301,8/1/2015 00:00:05,8/1/2015 00:05:06,450,W 49 St & 8 Ave,40.762272,-73.987882,479,9 Ave & W 45 St,40.760193,-73.991255,22675,Subscriber,1951.0,2
2,431,8/1/2015 00:00:06,8/1/2015 00:07:18,312,Allen St & E Houston St,40.722055,-73.989111,296,Division St & Bowery,40.714131,-73.997047,19831,Subscriber,1985.0,1
3,273,8/1/2015 00:00:09,8/1/2015 00:04:43,382,University Pl & E 14 St,40.734927,-73.992005,229,Great Jones St,40.727434,-73.993790,22765,Subscriber,1975.0,1
4,1256,8/1/2015 00:00:17,8/1/2015 00:21:13,352,W 56 St & 6 Ave,40.763406,-73.977225,432,E 7 St & Avenue A,40.726218,-73.983799,22127,Subscriber,1978.0,1


In [6]:
raw_data.shape

(1179044, 15)

In [7]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179044 entries, 0 to 1179043
Data columns (total 15 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   tripduration             1179044 non-null  int64  
 1   starttime                1179044 non-null  object 
 2   stoptime                 1179044 non-null  object 
 3   start station id         1179044 non-null  int64  
 4   start station name       1179044 non-null  object 
 5   start station latitude   1179044 non-null  float64
 6   start station longitude  1179044 non-null  float64
 7   end station id           1179044 non-null  int64  
 8   end station name         1179044 non-null  object 
 9   end station latitude     1179044 non-null  float64
 10  end station longitude    1179044 non-null  float64
 11  bikeid                   1179044 non-null  int64  
 12  usertype                 1179044 non-null  object 
 13  birth year               958043 non-null  

### <font color='red'>0.3) Load model from '../models/randomforest.pkl'</font>

In [8]:
import pickle
file_name = "randomforest.pkl"
#file_name = "logisticregression.pkl"
rf = pickle.load(open('/content/drive/My Drive/dataset_gclass/'+file_name, "rb"))

# 1) Data cleansing and feature engineering <font color='blue'> (30 min) </font>

### <font color='red'>1.1) Get rid of trips that are more than 1.5 hours long (watch out for variable units !)</font>

In [9]:
data = data[(data['tripduration']/3600) <= 1.5]

### <font color='red'>1.2) Impute missing values for birth year</font>

In [10]:
data['birth year'].fillna(data['birth year'].median(), inplace = True)

### <font color='red'>1.3) Create 'start_day' variable</font>

In [11]:
data.sample(3)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
175893,634,8/5/2015 18:15:30,8/5/2015 18:26:04,247,Perry St & Bleecker St,40.735354,-74.004831,237,E 11 St & 2 Ave,40.730473,-73.986724,23392,Subscriber,1988.0,2
321257,275,8/9/2015 13:43:33,8/9/2015 13:48:09,457,Broadway & W 58 St,40.766953,-73.981693,469,Broadway & W 53 St,40.763441,-73.982681,18131,Customer,1980.0,0
899554,744,8/24/2015 21:07:48,8/24/2015 21:20:13,476,E 31 St & 3 Ave,40.743943,-73.979661,441,E 52 St & 2 Ave,40.756014,-73.967416,14940,Subscriber,1984.0,1


In [12]:
import calendar
data['starttime'] = pd.to_datetime(data['starttime'], format="%m/%d/%Y %H:%M:%S")
data['stoptime'] = pd.to_datetime(data['stoptime'], format="%m/%d/%Y %H:%M:%S")
data['start_day'] = data.starttime.apply(lambda x: calendar.day_name[x.weekday()])

In [13]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start_day
0,1202,2015-08-01 00:00:04,2015-08-01 00:20:07,168,W 18 St & 6 Ave,40.739713,-73.994564,385,E 55 St & 2 Ave,40.757973,-73.966033,23253,Subscriber,1987.0,1,Saturday
1,301,2015-08-01 00:00:05,2015-08-01 00:05:06,450,W 49 St & 8 Ave,40.762272,-73.987882,479,9 Ave & W 45 St,40.760193,-73.991255,22675,Subscriber,1951.0,2,Saturday
2,431,2015-08-01 00:00:06,2015-08-01 00:07:18,312,Allen St & E Houston St,40.722055,-73.989111,296,Division St & Bowery,40.714131,-73.997047,19831,Subscriber,1985.0,1,Saturday
3,273,2015-08-01 00:00:09,2015-08-01 00:04:43,382,University Pl & E 14 St,40.734927,-73.992005,229,Great Jones St,40.727434,-73.993790,22765,Subscriber,1975.0,1,Saturday
4,1256,2015-08-01 00:00:17,2015-08-01 00:21:13,352,W 56 St & 6 Ave,40.763406,-73.977225,432,E 7 St & Avenue A,40.726218,-73.983799,22127,Subscriber,1978.0,1,Saturday


### <font color='red'>1.4) Create 'is_weekend' variable</font>

In [14]:
data['is_weekend'] = data.start_day.apply(lambda x: 1 if (x == 'Saturday' or x == 'Sunday') else 0)

In [15]:
data

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start_day,is_weekend
0,1202,2015-08-01 00:00:04,2015-08-01 00:20:07,168,W 18 St & 6 Ave,40.739713,-73.994564,385,E 55 St & 2 Ave,40.757973,-73.966033,23253,Subscriber,1987.0,1,Saturday,1
1,301,2015-08-01 00:00:05,2015-08-01 00:05:06,450,W 49 St & 8 Ave,40.762272,-73.987882,479,9 Ave & W 45 St,40.760193,-73.991255,22675,Subscriber,1951.0,2,Saturday,1
2,431,2015-08-01 00:00:06,2015-08-01 00:07:18,312,Allen St & E Houston St,40.722055,-73.989111,296,Division St & Bowery,40.714131,-73.997047,19831,Subscriber,1985.0,1,Saturday,1
3,273,2015-08-01 00:00:09,2015-08-01 00:04:43,382,University Pl & E 14 St,40.734927,-73.992005,229,Great Jones St,40.727434,-73.993790,22765,Subscriber,1975.0,1,Saturday,1
4,1256,2015-08-01 00:00:17,2015-08-01 00:21:13,352,W 56 St & 6 Ave,40.763406,-73.977225,432,E 7 St & Avenue A,40.726218,-73.983799,22127,Subscriber,1978.0,1,Saturday,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179039,423,2015-08-31 23:58:43,2015-09-01 00:05:47,3119,Vernon Blvd & 50 Ave,40.742327,-73.954117,3118,McGuinness Blvd & Eagle St,40.735550,-73.952840,18686,Subscriber,1988.0,1,Monday,0
1179040,1021,2015-08-31 23:58:54,2015-09-01 00:15:56,518,E 39 St & 2 Ave,40.747804,-73.973442,348,W Broadway & Spring St,40.724910,-74.001547,23487,Subscriber,1970.0,1,Monday,0
1179041,1896,2015-08-31 23:59:11,2015-09-01 00:30:47,336,Sullivan St & Washington Sq,40.730477,-73.999061,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,23509,Subscriber,1982.0,1,Monday,0
1179042,826,2015-08-31 23:59:16,2015-09-01 00:13:02,3075,Division Ave & Marcy Ave,40.707087,-73.957968,364,Lafayette Ave & Classon Ave,40.689004,-73.960239,15405,Customer,1980.0,0,Monday,0


### <font color='red'>1.5) Create 'start_moment' variable</font>

In [16]:
def time_of_day(x):
    if x.hour < 6 or x.hour >= 22:    #### COMPLETE THE FUNCTION BELOW ####
        return 'night'
    elif x.hour > 18 and x.hour < 22:
        return 'evening'
    elif x.hour >= 12 and x.hour <= 18:
        return 'afternoon'
    else:
        return 'morning'
data['start_moment'] = data.starttime.apply(time_of_day)

In [17]:
data

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start_day,is_weekend,start_moment
0,1202,2015-08-01 00:00:04,2015-08-01 00:20:07,168,W 18 St & 6 Ave,40.739713,-73.994564,385,E 55 St & 2 Ave,40.757973,-73.966033,23253,Subscriber,1987.0,1,Saturday,1,night
1,301,2015-08-01 00:00:05,2015-08-01 00:05:06,450,W 49 St & 8 Ave,40.762272,-73.987882,479,9 Ave & W 45 St,40.760193,-73.991255,22675,Subscriber,1951.0,2,Saturday,1,night
2,431,2015-08-01 00:00:06,2015-08-01 00:07:18,312,Allen St & E Houston St,40.722055,-73.989111,296,Division St & Bowery,40.714131,-73.997047,19831,Subscriber,1985.0,1,Saturday,1,night
3,273,2015-08-01 00:00:09,2015-08-01 00:04:43,382,University Pl & E 14 St,40.734927,-73.992005,229,Great Jones St,40.727434,-73.993790,22765,Subscriber,1975.0,1,Saturday,1,night
4,1256,2015-08-01 00:00:17,2015-08-01 00:21:13,352,W 56 St & 6 Ave,40.763406,-73.977225,432,E 7 St & Avenue A,40.726218,-73.983799,22127,Subscriber,1978.0,1,Saturday,1,night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179039,423,2015-08-31 23:58:43,2015-09-01 00:05:47,3119,Vernon Blvd & 50 Ave,40.742327,-73.954117,3118,McGuinness Blvd & Eagle St,40.735550,-73.952840,18686,Subscriber,1988.0,1,Monday,0,night
1179040,1021,2015-08-31 23:58:54,2015-09-01 00:15:56,518,E 39 St & 2 Ave,40.747804,-73.973442,348,W Broadway & Spring St,40.724910,-74.001547,23487,Subscriber,1970.0,1,Monday,0,night
1179041,1896,2015-08-31 23:59:11,2015-09-01 00:30:47,336,Sullivan St & Washington Sq,40.730477,-73.999061,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,23509,Subscriber,1982.0,1,Monday,0,night
1179042,826,2015-08-31 23:59:16,2015-09-01 00:13:02,3075,Division Ave & Marcy Ave,40.707087,-73.957968,364,Lafayette Ave & Classon Ave,40.689004,-73.960239,15405,Customer,1980.0,0,Monday,0,night


### <font color='red'>1.6) Create 'is_circle_trip' variable</font>

In [18]:
data['is_circle_trip'] = data.apply(lambda x: 1 if x['start station id'] == x['end station id'] else 0, axis = 1)

In [19]:
data

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start_day,is_weekend,start_moment,is_circle_trip
0,1202,2015-08-01 00:00:04,2015-08-01 00:20:07,168,W 18 St & 6 Ave,40.739713,-73.994564,385,E 55 St & 2 Ave,40.757973,-73.966033,23253,Subscriber,1987.0,1,Saturday,1,night,0
1,301,2015-08-01 00:00:05,2015-08-01 00:05:06,450,W 49 St & 8 Ave,40.762272,-73.987882,479,9 Ave & W 45 St,40.760193,-73.991255,22675,Subscriber,1951.0,2,Saturday,1,night,0
2,431,2015-08-01 00:00:06,2015-08-01 00:07:18,312,Allen St & E Houston St,40.722055,-73.989111,296,Division St & Bowery,40.714131,-73.997047,19831,Subscriber,1985.0,1,Saturday,1,night,0
3,273,2015-08-01 00:00:09,2015-08-01 00:04:43,382,University Pl & E 14 St,40.734927,-73.992005,229,Great Jones St,40.727434,-73.993790,22765,Subscriber,1975.0,1,Saturday,1,night,0
4,1256,2015-08-01 00:00:17,2015-08-01 00:21:13,352,W 56 St & 6 Ave,40.763406,-73.977225,432,E 7 St & Avenue A,40.726218,-73.983799,22127,Subscriber,1978.0,1,Saturday,1,night,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179039,423,2015-08-31 23:58:43,2015-09-01 00:05:47,3119,Vernon Blvd & 50 Ave,40.742327,-73.954117,3118,McGuinness Blvd & Eagle St,40.735550,-73.952840,18686,Subscriber,1988.0,1,Monday,0,night,0
1179040,1021,2015-08-31 23:58:54,2015-09-01 00:15:56,518,E 39 St & 2 Ave,40.747804,-73.973442,348,W Broadway & Spring St,40.724910,-74.001547,23487,Subscriber,1970.0,1,Monday,0,night,0
1179041,1896,2015-08-31 23:59:11,2015-09-01 00:30:47,336,Sullivan St & Washington Sq,40.730477,-73.999061,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,23509,Subscriber,1982.0,1,Monday,0,night,0
1179042,826,2015-08-31 23:59:16,2015-09-01 00:13:02,3075,Division Ave & Marcy Ave,40.707087,-73.957968,364,Lafayette Ave & Classon Ave,40.689004,-73.960239,15405,Customer,1980.0,0,Monday,0,night,0


### <font color='red'>1.7) Create 'june_day' variable</font>

In [20]:
data['june_day'] = data.starttime.apply(lambda x: x.day)

In [21]:
data

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start_day,is_weekend,start_moment,is_circle_trip,june_day
0,1202,2015-08-01 00:00:04,2015-08-01 00:20:07,168,W 18 St & 6 Ave,40.739713,-73.994564,385,E 55 St & 2 Ave,40.757973,-73.966033,23253,Subscriber,1987.0,1,Saturday,1,night,0,1
1,301,2015-08-01 00:00:05,2015-08-01 00:05:06,450,W 49 St & 8 Ave,40.762272,-73.987882,479,9 Ave & W 45 St,40.760193,-73.991255,22675,Subscriber,1951.0,2,Saturday,1,night,0,1
2,431,2015-08-01 00:00:06,2015-08-01 00:07:18,312,Allen St & E Houston St,40.722055,-73.989111,296,Division St & Bowery,40.714131,-73.997047,19831,Subscriber,1985.0,1,Saturday,1,night,0,1
3,273,2015-08-01 00:00:09,2015-08-01 00:04:43,382,University Pl & E 14 St,40.734927,-73.992005,229,Great Jones St,40.727434,-73.993790,22765,Subscriber,1975.0,1,Saturday,1,night,0,1
4,1256,2015-08-01 00:00:17,2015-08-01 00:21:13,352,W 56 St & 6 Ave,40.763406,-73.977225,432,E 7 St & Avenue A,40.726218,-73.983799,22127,Subscriber,1978.0,1,Saturday,1,night,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179039,423,2015-08-31 23:58:43,2015-09-01 00:05:47,3119,Vernon Blvd & 50 Ave,40.742327,-73.954117,3118,McGuinness Blvd & Eagle St,40.735550,-73.952840,18686,Subscriber,1988.0,1,Monday,0,night,0,31
1179040,1021,2015-08-31 23:58:54,2015-09-01 00:15:56,518,E 39 St & 2 Ave,40.747804,-73.973442,348,W Broadway & Spring St,40.724910,-74.001547,23487,Subscriber,1970.0,1,Monday,0,night,0,31
1179041,1896,2015-08-31 23:59:11,2015-09-01 00:30:47,336,Sullivan St & Washington Sq,40.730477,-73.999061,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,23509,Subscriber,1982.0,1,Monday,0,night,0,31
1179042,826,2015-08-31 23:59:16,2015-09-01 00:13:02,3075,Division Ave & Marcy Ave,40.707087,-73.957968,364,Lafayette Ave & Classon Ave,40.689004,-73.960239,15405,Customer,1980.0,0,Monday,0,night,0,31


### <font color='red'>1.8) Create 'traveled_distance' variable</font>

In [22]:
!pip install haversine

In [23]:
from haversine import haversine

def distance_stations(x):
    start_lat = x['start station latitude']
    start_long = x['start station longitude']
    end_lat = x['end station latitude']
    end_long = x['end station longitude']
    return haversine((start_lat,start_long),(end_lat,end_long))

data['traveled_distance'] = data.apply(distance_stations, axis=1)

In [24]:
data

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start_day,is_weekend,start_moment,is_circle_trip,june_day,traveled_distance
0,1202,2015-08-01 00:00:04,2015-08-01 00:20:07,168,W 18 St & 6 Ave,40.739713,-73.994564,385,E 55 St & 2 Ave,40.757973,-73.966033,23253,Subscriber,1987.0,1,Saturday,1,night,0,1,3.146289
1,301,2015-08-01 00:00:05,2015-08-01 00:05:06,450,W 49 St & 8 Ave,40.762272,-73.987882,479,9 Ave & W 45 St,40.760193,-73.991255,22675,Subscriber,1951.0,2,Saturday,1,night,0,1,0.366300
2,431,2015-08-01 00:00:06,2015-08-01 00:07:18,312,Allen St & E Houston St,40.722055,-73.989111,296,Division St & Bowery,40.714131,-73.997047,19831,Subscriber,1985.0,1,Saturday,1,night,0,1,1.106204
3,273,2015-08-01 00:00:09,2015-08-01 00:04:43,382,University Pl & E 14 St,40.734927,-73.992005,229,Great Jones St,40.727434,-73.993790,22765,Subscriber,1975.0,1,Saturday,1,night,0,1,0.846623
4,1256,2015-08-01 00:00:17,2015-08-01 00:21:13,352,W 56 St & 6 Ave,40.763406,-73.977225,432,E 7 St & Avenue A,40.726218,-73.983799,22127,Subscriber,1978.0,1,Saturday,1,night,0,1,4.172070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179039,423,2015-08-31 23:58:43,2015-09-01 00:05:47,3119,Vernon Blvd & 50 Ave,40.742327,-73.954117,3118,McGuinness Blvd & Eagle St,40.735550,-73.952840,18686,Subscriber,1988.0,1,Monday,0,night,0,31,0.761265
1179040,1021,2015-08-31 23:58:54,2015-09-01 00:15:56,518,E 39 St & 2 Ave,40.747804,-73.973442,348,W Broadway & Spring St,40.724910,-74.001547,23487,Subscriber,1970.0,1,Monday,0,night,0,31,3.476766
1179041,1896,2015-08-31 23:59:11,2015-09-01 00:30:47,336,Sullivan St & Washington Sq,40.730477,-73.999061,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,23509,Subscriber,1982.0,1,Monday,0,night,0,31,5.530142
1179042,826,2015-08-31 23:59:16,2015-09-01 00:13:02,3075,Division Ave & Marcy Ave,40.707087,-73.957968,364,Lafayette Ave & Classon Ave,40.689004,-73.960239,15405,Customer,1980.0,0,Monday,0,night,0,31,2.019786


### <font color='red'>1.9) Create 'average_speed' variable</font>

In [25]:
data['average_speed'] = data.apply(lambda x: x['traveled_distance']/(x['tripduration']/3600), axis=1)

In [26]:
data

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start_day,is_weekend,start_moment,is_circle_trip,june_day,traveled_distance,average_speed
0,1202,2015-08-01 00:00:04,2015-08-01 00:20:07,168,W 18 St & 6 Ave,40.739713,-73.994564,385,E 55 St & 2 Ave,40.757973,-73.966033,23253,Subscriber,1987.0,1,Saturday,1,night,0,1,3.146289,9.423161
1,301,2015-08-01 00:00:05,2015-08-01 00:05:06,450,W 49 St & 8 Ave,40.762272,-73.987882,479,9 Ave & W 45 St,40.760193,-73.991255,22675,Subscriber,1951.0,2,Saturday,1,night,0,1,0.366300,4.380996
2,431,2015-08-01 00:00:06,2015-08-01 00:07:18,312,Allen St & E Houston St,40.722055,-73.989111,296,Division St & Bowery,40.714131,-73.997047,19831,Subscriber,1985.0,1,Saturday,1,night,0,1,1.106204,9.239752
3,273,2015-08-01 00:00:09,2015-08-01 00:04:43,382,University Pl & E 14 St,40.734927,-73.992005,229,Great Jones St,40.727434,-73.993790,22765,Subscriber,1975.0,1,Saturday,1,night,0,1,0.846623,11.164264
4,1256,2015-08-01 00:00:17,2015-08-01 00:21:13,352,W 56 St & 6 Ave,40.763406,-73.977225,432,E 7 St & Avenue A,40.726218,-73.983799,22127,Subscriber,1978.0,1,Saturday,1,night,0,1,4.172070,11.958161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179039,423,2015-08-31 23:58:43,2015-09-01 00:05:47,3119,Vernon Blvd & 50 Ave,40.742327,-73.954117,3118,McGuinness Blvd & Eagle St,40.735550,-73.952840,18686,Subscriber,1988.0,1,Monday,0,night,0,31,0.761265,6.478851
1179040,1021,2015-08-31 23:58:54,2015-09-01 00:15:56,518,E 39 St & 2 Ave,40.747804,-73.973442,348,W Broadway & Spring St,40.724910,-74.001547,23487,Subscriber,1970.0,1,Monday,0,night,0,31,3.476766,12.258920
1179041,1896,2015-08-31 23:59:11,2015-09-01 00:30:47,336,Sullivan St & Washington Sq,40.730477,-73.999061,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,23509,Subscriber,1982.0,1,Monday,0,night,0,31,5.530142,10.500269
1179042,826,2015-08-31 23:59:16,2015-09-01 00:13:02,3075,Division Ave & Marcy Ave,40.707087,-73.957968,364,Lafayette Ave & Classon Ave,40.689004,-73.960239,15405,Customer,1980.0,0,Monday,0,night,0,31,2.019786,8.802941


### <font color='red'>1.10) Variable encoding</font>

In [27]:
for variable_name in ['start_day','is_weekend',
                      'start_moment','is_circle_trip']:
    print('Dummifying the {} variable ...'.format(variable_name))
    
    dummies = pd.get_dummies(data[variable_name])
    #### CREATE DUMMIES FROM THE COLUMN data[variable_name], as you did above with pd.get_dummies() ####
    
    dummies.columns = ['{}_{}'.format(variable_name,x) for x in dummies.columns]  # this will rename the column
                                                                                  # in an appropriate way
        
    data = pd.concat([data,dummies],axis=1)  # This will append the dummy column to the existing dataframe
    

#### CHECK THAT DUMMIFICATION IS SUCCESSFUL BY PRINTING SAMPLES AND COLUMNS NAMES ####
dummy = ['start_day_Friday',
       'start_day_Monday', 'start_day_Saturday', 'start_day_Sunday',
       'start_day_Thursday', 'start_day_Tuesday', 'start_day_Wednesday',
       'is_weekend_0', 'is_weekend_1', 'start_moment_afternoon',
       'start_moment_evening', 'start_moment_morning', 'start_moment_night',
       'is_circle_trip_0', 'is_circle_trip_1']
data[dummy].sample(2)

Dummifying the start_day variable ...
Dummifying the is_weekend variable ...
Dummifying the start_moment variable ...
Dummifying the is_circle_trip variable ...


,start_day_Friday,start_day_Monday,start_day_Saturday,start_day_Sunday,start_day_Thursday,start_day_Tuesday,start_day_Wednesday,is_weekend_0,is_weekend_1,start_moment_afternoon,start_moment_evening,start_moment_morning,start_moment_night,is_circle_trip_0,is_circle_trip_1
546696,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0
938884,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0


In [28]:
for variable_name in ['start_day','is_weekend',
                      'start_moment','is_circle_trip']:
    print('Deleting the {} variable ...'.format(variable_name))
    #### DELETE THE OLD COLUMN USING del data[column_name_here] HERE ####
    del data[variable_name]

Deleting the start_day variable ...
Deleting the is_weekend variable ...
Deleting the start_moment variable ...
Deleting the is_circle_trip variable ...


In [29]:
data

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,june_day,traveled_distance,average_speed,start_day_Friday,start_day_Monday,start_day_Saturday,start_day_Sunday,start_day_Thursday,start_day_Tuesday,start_day_Wednesday,is_weekend_0,is_weekend_1,start_moment_afternoon,start_moment_evening,start_moment_morning,start_moment_night,is_circle_trip_0,is_circle_trip_1
0,1202,2015-08-01 00:00:04,2015-08-01 00:20:07,168,W 18 St & 6 Ave,40.739713,-73.994564,385,E 55 St & 2 Ave,40.757973,-73.966033,23253,Subscriber,1987.0,1,1,3.146289,9.423161,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
1,301,2015-08-01 00:00:05,2015-08-01 00:05:06,450,W 49 St & 8 Ave,40.762272,-73.987882,479,9 Ave & W 45 St,40.760193,-73.991255,22675,Subscriber,1951.0,2,1,0.366300,4.380996,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
2,431,2015-08-01 00:00:06,2015-08-01 00:07:18,312,Allen St & E Houston St,40.722055,-73.989111,296,Division St & Bowery,40.714131,-73.997047,19831,Subscriber,1985.0,1,1,1.106204,9.239752,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
3,273,2015-08-01 00:00:09,2015-08-01 00:04:43,382,University Pl & E 14 St,40.734927,-73.992005,229,Great Jones St,40.727434,-73.993790,22765,Subscriber,1975.0,1,1,0.846623,11.164264,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
4,1256,2015-08-01 00:00:17,2015-08-01 00:21:13,352,W 56 St & 6 Ave,40.763406,-73.977225,432,E 7 St & Avenue A,40.726218,-73.983799,22127,Subscriber,1978.0,1,1,4.172070,11.958161,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179039,423,2015-08-31 23:58:43,2015-09-01 00:05:47,3119,Vernon Blvd & 50 Ave,40.742327,-73.954117,3118,McGuinness Blvd & Eagle St,40.735550,-73.952840,18686,Subscriber,1988.0,1,31,0.761265,6.478851,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0
1179040,1021,2015-08-31 23:58:54,2015-09-01 00:15:56,518,E 39 St & 2 Ave,40.747804,-73.973442,348,W Broadway & Spring St,40.724910,-74.001547,23487,Subscriber,1970.0,1,31,3.476766,12.258920,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0
1179041,1896,2015-08-31 23:59:11,2015-09-01 00:30:47,336,Sullivan St & Washington Sq,40.730477,-73.999061,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,23509,Subscriber,1982.0,1,31,5.530142,10.500269,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0
1179042,826,2015-08-31 23:59:16,2015-09-01 00:13:02,3075,Division Ave & Marcy Ave,40.707087,-73.957968,364,Lafayette Ave & Classon Ave,40.689004,-73.960239,15405,Customer,1980.0,0,31,2.019786,8.802941,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0


### <font color='red'>1.11) Remove unnecessary columns</font>

In [30]:
del data['starttime'], data['stoptime'], data['start station name'], data['end station name']
del data['gender'], data['birth year']
del data['bikeid']
del data['start station id'], data['end station id']

In [31]:
data

,tripduration,start station latitude,start station longitude,end station latitude,end station longitude,usertype,june_day,traveled_distance,average_speed,start_day_Friday,start_day_Monday,start_day_Saturday,start_day_Sunday,start_day_Thursday,start_day_Tuesday,start_day_Wednesday,is_weekend_0,is_weekend_1,start_moment_afternoon,start_moment_evening,start_moment_morning,start_moment_night,is_circle_trip_0,is_circle_trip_1
0,1202,40.739713,-73.994564,40.757973,-73.966033,Subscriber,1,3.146289,9.423161,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
1,301,40.762272,-73.987882,40.760193,-73.991255,Subscriber,1,0.366300,4.380996,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
2,431,40.722055,-73.989111,40.714131,-73.997047,Subscriber,1,1.106204,9.239752,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
3,273,40.734927,-73.992005,40.727434,-73.993790,Subscriber,1,0.846623,11.164264,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
4,1256,40.763406,-73.977225,40.726218,-73.983799,Subscriber,1,4.172070,11.958161,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179039,423,40.742327,-73.954117,40.735550,-73.952840,Subscriber,31,0.761265,6.478851,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0
1179040,1021,40.747804,-73.973442,40.724910,-74.001547,Subscriber,31,3.476766,12.258920,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0
1179041,1896,40.730477,-73.999061,40.683826,-73.976323,Subscriber,31,5.530142,10.500269,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0
1179042,826,40.707087,-73.957968,40.689004,-73.960239,Customer,31,2.019786,8.802941,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0


### <font color='red'>1.12) Build arrays for features</font>

In [32]:
import sklearn
del data['usertype']
features = np.array(data)

In [33]:
features

array([[ 1.20200000e+03,  4.07397130e+01, -7.39945640e+01, ...,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 3.01000000e+02,  4.07622720e+01, -7.39878820e+01, ...,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 4.31000000e+02,  4.07220550e+01, -7.39891110e+01, ...,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.89600000e+03,  4.07304775e+01, -7.39990607e+01, ...,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 8.26000000e+02,  4.07070870e+01, -7.39579678e+01, ...,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 1.05000000e+02,  4.07439431e+01, -7.39796607e+01, ...,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00]])

# 2) Make Predictions <font color='blue'> (10 min) </font>

In [34]:
predictions = rf.predict(features)

In [35]:
predictions

array([1, 1, 1, ..., 1, 1, 1])

# 3) Final Exam <font color='blue'> (60 min) </font>

In [36]:
data.head()

,tripduration,start station latitude,start station longitude,end station latitude,end station longitude,june_day,traveled_distance,average_speed,start_day_Friday,start_day_Monday,start_day_Saturday,start_day_Sunday,start_day_Thursday,start_day_Tuesday,start_day_Wednesday,is_weekend_0,is_weekend_1,start_moment_afternoon,start_moment_evening,start_moment_morning,start_moment_night,is_circle_trip_0,is_circle_trip_1
0,1202,40.739713,-73.994564,40.757973,-73.966033,1,3.146289,9.423161,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
1,301,40.762272,-73.987882,40.760193,-73.991255,1,0.366300,4.380996,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
2,431,40.722055,-73.989111,40.714131,-73.997047,1,1.106204,9.239752,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
3,273,40.734927,-73.992005,40.727434,-73.993790,1,0.846623,11.164264,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
4,1256,40.763406,-73.977225,40.726218,-73.983799,1,4.172070,11.958161,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0


In [37]:
data.columns

Index(['tripduration', 'start station latitude', 'start station longitude',
       'end station latitude', 'end station longitude', 'june_day',
       'traveled_distance', 'average_speed', 'start_day_Friday',
       'start_day_Monday', 'start_day_Saturday', 'start_day_Sunday',
       'start_day_Thursday', 'start_day_Tuesday', 'start_day_Wednesday',
       'is_weekend_0', 'is_weekend_1', 'start_moment_afternoon',
       'start_moment_evening', 'start_moment_morning', 'start_moment_night',
       'is_circle_trip_0', 'is_circle_trip_1'],
      dtype='object')

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1169041 entries, 0 to 1179043
Data columns (total 23 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   tripduration             1169041 non-null  int64  
 1   start station latitude   1169041 non-null  float64
 2   start station longitude  1169041 non-null  float64
 3   end station latitude     1169041 non-null  float64
 4   end station longitude    1169041 non-null  float64
 5   june_day                 1169041 non-null  int64  
 6   traveled_distance        1169041 non-null  float64
 7   average_speed            1169041 non-null  float64
 8   start_day_Friday         1169041 non-null  uint8  
 9   start_day_Monday         1169041 non-null  uint8  
 10  start_day_Saturday       1169041 non-null  uint8  
 11  start_day_Sunday         1169041 non-null  uint8  
 12  start_day_Thursday       1169041 non-null  uint8  
 13  start_day_Tuesday        1169041 non-null 

In [39]:
x = data[['start_day_Friday', 'start_day_Monday', 'start_day_Saturday', 'start_day_Sunday', 'start_day_Thursday', 'start_day_Tuesday','start_day_Wednesday']] .values  #.astype()
x[0:7]

array([[0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0]], dtype=uint8)

In [40]:
y = data['is_weekend_1']
y[0:10]

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: is_weekend_1, dtype: uint8

In [41]:
from sklearn import preprocessing
x = preprocessing.StandardScaler().fit(x).transform(x.astype(float))
x[0:7]

array([[-0.39794731, -0.44609601,  2.36118226, -0.39585372, -0.40664435,
        -0.37862909, -0.4076123 ],
       [-0.39794731, -0.44609601,  2.36118226, -0.39585372, -0.40664435,
        -0.37862909, -0.4076123 ],
       [-0.39794731, -0.44609601,  2.36118226, -0.39585372, -0.40664435,
        -0.37862909, -0.4076123 ],
       [-0.39794731, -0.44609601,  2.36118226, -0.39585372, -0.40664435,
        -0.37862909, -0.4076123 ],
       [-0.39794731, -0.44609601,  2.36118226, -0.39585372, -0.40664435,
        -0.37862909, -0.4076123 ],
       [-0.39794731, -0.44609601,  2.36118226, -0.39585372, -0.40664435,
        -0.37862909, -0.4076123 ],
       [-0.39794731, -0.44609601,  2.36118226, -0.39585372, -0.40664435,
        -0.37862909, -0.4076123 ]])

In [42]:
x = data.iloc[:,0:6].values
y = data.iloc[0:6].values
print(x)
print(y)

[[ 1.20200000e+03  4.07397130e+01 -7.39945640e+01  4.07579732e+01
  -7.39660331e+01  1.00000000e+00]
 [ 3.01000000e+02  4.07622720e+01 -7.39878820e+01  4.07601925e+01
  -7.39912551e+01  1.00000000e+00]
 [ 4.31000000e+02  4.07220550e+01 -7.39891110e+01  4.07141309e+01
  -7.39970468e+01  1.00000000e+00]
 ...
 [ 1.89600000e+03  4.07304775e+01 -7.39990607e+01  4.06838260e+01
  -7.39763233e+01  3.10000000e+01]
 [ 8.26000000e+02  4.07070870e+01 -7.39579678e+01  4.06890044e+01
  -7.39602385e+01  3.10000000e+01]
 [ 1.05000000e+02  4.07439431e+01 -7.39796607e+01  4.07444492e+01
  -7.39830353e+01  3.10000000e+01]]
[[ 1.20200000e+03  4.07397130e+01 -7.39945640e+01  4.07579732e+01
  -7.39660331e+01  1.00000000e+00  3.14628884e+00  9.42316126e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 3.01

In [43]:
x

array([[ 1.20200000e+03,  4.07397130e+01, -7.39945640e+01,
         4.07579732e+01, -7.39660331e+01,  1.00000000e+00],
       [ 3.01000000e+02,  4.07622720e+01, -7.39878820e+01,
         4.07601925e+01, -7.39912551e+01,  1.00000000e+00],
       [ 4.31000000e+02,  4.07220550e+01, -7.39891110e+01,
         4.07141309e+01, -7.39970468e+01,  1.00000000e+00],
       ...,
       [ 1.89600000e+03,  4.07304775e+01, -7.39990607e+01,
         4.06838260e+01, -7.39763233e+01,  3.10000000e+01],
       [ 8.26000000e+02,  4.07070870e+01, -7.39579678e+01,
         4.06890044e+01, -7.39602385e+01,  3.10000000e+01],
       [ 1.05000000e+02,  4.07439431e+01, -7.39796607e+01,
         4.07444492e+01, -7.39830353e+01,  3.10000000e+01]])

In [44]:
y

array([[ 1.20200000e+03,  4.07397130e+01, -7.39945640e+01,
         4.07579732e+01, -7.39660331e+01,  1.00000000e+00,
         3.14628884e+00,  9.42316126e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [ 3.01000000e+02,  4.07622720e+01, -7.39878820e+01,
         4.07601925e+01, -7.39912551e+01,  1.00000000e+00,
         3.66299961e-01,  4.38099621e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [ 4.31000000e+02,  4.07220550e+01, -7.39891110e+01,
         4.07141309e+01, -7.3

In [46]:
data.isna().sum()

tripduration               0
start station latitude     0
start station longitude    0
end station latitude       0
end station longitude      0
june_day                   0
traveled_distance          0
average_speed              0
start_day_Friday           0
start_day_Monday           0
start_day_Saturday         0
start_day_Sunday           0
start_day_Thursday         0
start_day_Tuesday          0
start_day_Wednesday        0
is_weekend_0               0
is_weekend_1               0
start_moment_afternoon     0
start_moment_evening       0
start_moment_morning       0
start_moment_night         0
is_circle_trip_0           0
is_circle_trip_1           0
dtype: int64